# Deep Averageing Netrok Analysis

In [50]:
import torch

embd = torch.nn.Embedding(20, 5)
x = torch.tensor([[1, 2, 3], [0, 2, 4]], dtype=torch.int32)
print(x)
out = embd(x)
print(out)
print(out.shape)
print(out.sum(axis=1))
print(out.sum(axis=1).shape)
lenthes = torch.tensor([3, 2])
print(lenthes.shape)
out.sum(axis=1)/lenthes.reshape([-1, 1])

tensor([[1, 2, 3],
        [0, 2, 4]], dtype=torch.int32)
tensor([[[ 1.0482,  2.0123, -0.5793, -1.6339,  0.5679],
         [-1.3865,  0.5137,  0.2737, -0.3556,  0.5181],
         [-1.4103, -0.2406, -1.9688, -0.3529,  1.5662]],

        [[-0.9058, -1.4235,  0.4452,  0.2892,  1.4724],
         [-1.3865,  0.5137,  0.2737, -0.3556,  0.5181],
         [ 1.4639,  0.1712, -0.5358,  1.1458,  0.2673]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([2, 3, 5])
tensor([[-1.7486,  2.2854, -2.2744, -2.3424,  2.6523],
        [-0.8284, -0.7387,  0.1831,  1.0793,  2.2579]], grad_fn=<SumBackward1>)
torch.Size([2, 5])
torch.Size([2])


tensor([[-0.5829,  0.7618, -0.7581, -0.7808,  0.8841],
        [-0.4142, -0.3693,  0.0916,  0.5397,  1.1289]], grad_fn=<DivBackward0>)

In [72]:
out = torch.randn(10, 2)
labels = torch.randint(0, 2,size=(10,))
labels
topval, topi = out.topk(1, dim=1)
true_vals = labels[labels == topi.reshape([10,])].shape[0]
error_vals = torch.nonzero(labels-topi.squeeze()).shape[0]
print('accuracy=', true_vals/labels.shape[0])
print('accuracy=', 1-(error_vals/labels.shape[0]))

accuracy= 0.6
accuracy= 0.6


# Training

## Training with no pretrained Embeddings

In [117]:
!python3  dan.py --num-epochs=100 --checkpoint=10 

2
DanModel(
  (embeddings): Embedding(12678, 50, padding_idx=0)
  (linear1): Linear(in_features=50, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=2, bias=True)
  (_softmax): Softmax(dim=1)
  (linear_stack): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=50, out_features=2, bias=True)
  )
)
start epoch 0
Saving Model ............
# of steps=10, Avg Train Loss=0.005377, Avg Dev Loss=0.005619, Train Acc=0.962358, Dev Acc=0.646865, Time: 1.948750
Saving Model ............
# of steps=20, Avg Train Loss=0.005284, Avg Dev Loss=0.005535, Train Acc=0.969531, Dev Acc=0.696370, Time: 2.142809
----------------------------------------------------------

start epoch 1
Saving Model ............
# of steps=10, Avg Train Loss=0.005159, Avg Dev Loss=0.005373, Train Acc=0.965909, Dev Acc=0.699670, Time: 0.323128
Saving Model ............
# of steps=20, Avg Train Loss=